# Viya 2020 Modeling in Python and SAS AutoML
**Notebook authored by Sophia Rowland (sophia.rowland@sas.com)**

In this simple example, we will build an automated machine learning model using SAS Data Science Pilot via the SWAT package and a Python model pipeline using xgboost and push both models into model manager using the sasctl and pzmm package.
***
## Table of Contents
1. [Introduction](#Introduction)
1. [Gather Resources](#Gather-Resources)
1. [Data Science Pilot in Python](#Data-Science-Pilot-in-Python)
1. [Python XGBOOST Machine Learning Pipeline](#Python-XGBOOST-Machine-Learning-Pipeline)
1. [Register Models to SAS Model Manager](#Register-Models-to-SAS-Model-Manager)
1. [Accessing Models via API](#Accessing-Models-via-API)
1. [Conclusion](#Conclusion)
***
## Introduction 
In this notebook, we will connect to SAS Viya from Python using the SWAT package. Using SWAT, we will use the Data Science Pilot action set to build automated machine learning models using the power of SAS analytics. Next, we will create a simple XGBOOST model pipeline, which will push into the SAS model repository service to demonstrate Viya's ability to consume different types of models! Finally, we will use the SASCTL and PZMM packages to generate model metadata and push our SAS models and our native Python model into the model repository service. More information about SWAT, Data Science Pilot, SASCTL and PZMM are available below.

### The Scripting Wrapper for Analytics Transfer Package
The [SAS SWAT (Scripting Wrapper for Analytics Transfer) package for Python](https://sassoftware.github.io/python-swat/index.html) is a Python interface to the SAS Cloud Analytic Services (CAS) engine (the centerpiece of the SAS Viya framework). SWAT allows Python programmers to load and access data stored in Viya as CASTables as well as execute CAS actions upon the data stored in CAS. CAS Action Sets are akin to libraries and CAS Actions can be thought of as functions. Examples and syntax for utilizing CAS actions in Python are readily available in the [documentation](https://go.documentation.sas.com/?cdcId=pgmsascdc&cdcVersion=v_006&docsetId=casactml&docsetTarget=titlepage.htm&locale=en). SWAT accesses the CAS Engine using our Open API Framework. Notice in the image below that programmers can interface with CASTables using a familiar pandas-like syntax. With the best-of-breed SAS analytics in the cloud and the use of Python and its large collection of open source packages, the SWAT package gives you access to the best of both worlds.

### The Data Science Pilot Action Set
The Data Science Pilot Action Set is included with SAS Visual Data Mining and Machine Learning (VDMML) and consists of actions that implement a policy-based, configurable, and scalable approach to automating data science workflows. These actions allow you to move from a data set to a deployable model in a flash! A deeper dive into the data science can be found in [my blog series](https://blogs.sas.com/content/tag/data-science-pilot-explained/).

### The SASCTL and PZMM Packages
The SASCTL and PZMM Packages are incredibly helpful with model metadata, management and governance. SASCTL and PZMM enable you to easily put models built using CAS actions or purely python into our model repository and save important metadata with those models. You can also publish the models for easily deployment, monitor model performance, or access our workflow service. SASCTL and PZMM access the model management and model deployment microservices using our open API framework.

Now that we have a better understanding of SAS Open Source packages, let's get started!
***
## Gather Resources 
First, we will import necessary packages.

In [1]:
# Packages for Python Basics
import sys
import numpy as np
import pandas as pd

# Packages for Building Model Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb

# Packages for the Environment
from pathlib import Path
import os
import urllib3
import getpass

# Functions for working with SAS and Python
from sasctl.services import model_repository as modelRepo 
from sasctl.tasks import register_model, publish_model, update_model_performance
from sasctl import pzmm as pzmm
from sasctl import Session
import swat

Now I will use the Scripting Wrapper for Analytics Transfer (SWAT) package to connect to out Cloud Analytics Service (CAS). To use this notebook yourself, ensure that you can connect to a SAS Viya 2020 environment and input your credentials below! 

In [2]:
# For authentication
username = getpass.getpass("Username: ")
password = getpass.getpass("Password: ")
hostname = getpass.getpass("Hostname: ")
portnum = int(getpass.getpass("Portnumber: "))
os.environ['CAS_CLIENT_SSL_CA_LIST']= getpass.getpass("SSL Certificate: ")

# Build connection
conn =  swat.CAS(hostname, portnum, username, password)

Username:  ······
Password:  ··············
Hostname:  ········································
Portnumber:  ·····
SSL Certificate:  ······················································································


Next, we will load the csv file holding our into a pandas data frame and into a CAS table. To use this notebook yourself, edit the code to point to a copy of the HMEQ data set. You can also use a copy residing on your SAS Viya 2020 environment! 

In [3]:
hmeqdf = pd.read_csv("./data/hmeq.csv")
hmeqct = conn.read_csv("./data/hmeq.csv", casout=dict(name='hmeqct', replace=True))

NOTE: Cloud Analytic Services made the uploaded file available as table HMEQCT in caslib CASUSER(sorowl).
NOTE: The table HMEQCT has been created in caslib CASUSER(sorowl) from binary data uploaded to Cloud Analytic Services.


In [4]:
hmeqdf.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,26800,46236.0,62711.0,DebtCon,Office,17.0,0.0,0.0,175.075058,1.0,22.0,33.059934
1,0,26900,74982.0,126972.0,DebtCon,Office,0.0,0.0,0.0,315.818911,0.0,23.0,38.325990
2,0,26900,67144.0,92923.0,DebtCon,Other,16.0,0.0,0.0,89.112173,1.0,17.0,32.791478
3,0,26900,45763.0,73797.0,DebtCon,Other,23.0,NaN,0.0,291.591682,1.0,29.0,39.370858
4,0,27000,144901.0,178093.0,DebtCon,ProfExe,7.0,0.0,0.0,331.113972,0.0,34.0,40.566552


In [5]:
hmeqct.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0.0,26800.0,46236.0,62711.0,DebtCon,Office,17.0,0.0,0.0,175.075058,1.0,22.0,33.059934
1,0.0,26900.0,74982.0,126972.0,DebtCon,Office,0.0,0.0,0.0,315.818911,0.0,23.0,38.325990
2,0.0,26900.0,67144.0,92923.0,DebtCon,Other,16.0,0.0,0.0,89.112173,1.0,17.0,32.791478
3,0.0,26900.0,45763.0,73797.0,DebtCon,Other,23.0,NaN,0.0,291.591682,1.0,29.0,39.370858
4,0.0,27000.0,144901.0,178093.0,DebtCon,ProfExe,7.0,0.0,0.0,331.113972,0.0,34.0,40.566552


***
## Data Science Pilot in Python 

Now we will interface with the Data Science Pilot Action Set to build the same model we can build in SAS in Python - demonstrating the ability to code in both Python and SAS, while getting the same results!

In [6]:
conn.builtins.loadactionset('dataSciencePilot')
conn.dataSciencePilot.dsAutoMl(
    table = 'hmeqct',
    target = 'BAD', 
    explorationPolicy = {'cardinality': {'lowMediumCutoff':40}}, 
    screenPolicy = {'missingPercentThreshold':35}, 
    selectionPolicy = {'topk':10},
    transformationPolicy = {'entropy': True, 'iqv': True, 'kurtosis': True, 'outlier': True},
    modelTypes              = ["decisionTree", "gradboost"],
    objective               = "AUC",
    sampleSize              = 10,
    topKPipelines           = 5,
    kFolds                  = 2,
    transformationOut       = {"name" : "TRANSFORMATION_OUT_PY", "replace" : True},
    featureOut              = {"name" : "FEATURE_OUT_PY", "replace" : True},
    pipelineOut             = {"name" : "PIPELINE_OUT_PY", "replace" : True},
    saveState               = {"modelNamePrefix" : "ASTORE_OUT_PY", "replace" : True, "topK":1} 
)

NOTE: Added action set 'dataSciencePilot'.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Added action set 'autotune'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'decisionTree'.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Added action set 'autotune'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'decisionTree'.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Added action set 'autotune'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'decisionTree'.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Added action set 'autotune'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'decisionTree'.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Added action set 'autotune'.
NOTE: Ea

,Descr,Value
0,Number of Tree Nodes,31.000000
1,Max Number of Branches,2.000000
2,Number of Levels,5.000000
3,Number of Leaves,16.000000
4,Number of Bins,100.000000
5,Minimum Size of Leaves,6.000000
6,Maximum Size of Leaves,2045.000000
7,Number of Variables,10.000000
8,Confidence Level for Pruning,0.250000
9,Number of Observations Used,5960.000000


Great! Now let's use our feature creation model and our grad boost model to score our data.

In [7]:
conn.loadactionset('astore')
model_astore = conn.CASTable("ASTORE_OUT_PY_gradBoost_1")
feature_astore = conn.CASTable("ASTORE_OUT_PY_fm_")
conn.score(
    table='hmeqct', 
    copyvars='BAD', 
    out = dict(name='feat_scored', replace=True),
    rstore='ASTORE_OUT_PY_fm_')
conn.score(
    table='feat_scored', 
    copyvars='BAD', 
    out = dict(name='hmeq_scored', replace=True), 
    rstore='ASTORE_OUT_PY_gradBoost_1')
hmeq_scored = conn.CASTable('hmeq_scored')
hmeq_scored = hmeq_scored[['BAD', 'P_BAD1']].to_frame()
hmeq_scored.BAD = hmeq_scored.BAD.astype(int)
hmeq_scored = hmeq_scored.rename(columns={'BAD': '0', 'P_BAD1': '0'})
hmeq_scored = pd.DataFrame(hmeq_scored)

NOTE: Added action set 'astore'.


*** 
## Python XGBOOST Machine Learning Pipeline
Next, we will create a pipeline which will take the input variables, impute missing categorical variables, one-hot-encode categorical variables, and train a xgboost model.

In [8]:
# Separate target from data preparation process
y = hmeqdf.pop('BAD').values
# Find and notes categorical columns
kinds = np.array([dt.kind for dt in hmeqdf.dtypes])
all_columns = hmeqdf.columns.values
is_cat = kinds == 'O'
cat_cols = all_columns[is_cat]
# Impute missing categorical variables with missing
cat_si_step = ('si', SimpleImputer(strategy='constant', fill_value='MISSING'))
# One hot-encode categorical variables
cat_ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
# Create ML pipe with imputation, one-hot-ecndcoding, and xgboost
cat_steps = [cat_si_step, cat_ohe_step]
cat_pipe = Pipeline(cat_steps)
cat_transformers = [('cat', cat_pipe, cat_cols)]
ct = ColumnTransformer(transformers=cat_transformers)
ml_pipe = Pipeline([('transform', ct), ('xgb', xgb.XGBClassifier())])
ml_pipe.fit(hmeqdf, y)

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('si',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  array(['REASON', 'JOB'], dtype=object))])),
                ('xgb',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsampl..., gpu_id=-1,
                               importance_ty

This is a simple example, but let’s examine our model’s accuracy as a sanity check.

In [9]:
ml_pipe.score(hmeqdf, y)

0.8013422818791947

We now have a working pipeline. Now let's create some metadata around our models and push them into Model Manager.
***
## Register Models into SAS Model Manager
Our next-to-last step in this notebook is to register the models into SAS Model Manager. We are using the sasctl package to make this process easier. To use sasctl, we do need to start a session using our Viya credentials. Next, we use the register model function with our model, a name for our model, the input data set, and a name for the project.
### SWAT Model Management
Below is a quick function definition I created to make generate performance metadata for our model built using SWAT as well as push the model into SAS Model Manager.

In [10]:
def mm_swat(project_name, model_name, model_astore, feature_astore, feature_name, scored_data, target):
    ##################################################
    # CREATE OUTPUT INFORMATION  

    # Output folder name
    output_folder = 'Outputs'

    # Create Folder
    output_path = Path.cwd() / output_folder / model_name
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    ##################################################
    # CONNECT TO SAS VIYA
    host_session = 'https://' + hostname + '/'

    # Connect using a session 
    sess=Session(host_session, username, password, verify_ssl=False, protocol="http")

    ##################################################
    # COMPILE METADATA

    # Create JSON Files Object
    JSONFiles = pzmm.JSONFiles()    
    
    # Write Fit Statistics JSON
    JSONFiles.calculateFitStat(trainData=scored_data, jPath=output_path)

    # Write ROC amd Lift Statistics JSON
    JSONFiles.generateROCLiftStat(target, 1, conn, trainData=scored_data, jPath=output_path)
        
    model_content = [output_path / f for f in ('dmcas_fitstat.json', 'dmcas_lift.json', 'dmcas_roc.json')]
    model_content = {f.name: f for f in model_content}

    # Pushing Astore into Model Manager
    register_model(feature_astore, feature_name, project=project_name, force=True, version='latest')
    register_model(model_astore, model_name, project=project_name, force=True, version='latest', files=model_content) 

In [11]:
urllib3.disable_warnings()
# mm_swat(project_name, model_name, model_astore, feature_astore, feature_name, scored_data, target)
mm_swat('HMEQ Loan Default', 'SWAT_DSP_GRADBOOST', model_astore, feature_astore, 'SWAT_DSP_FEATS', hmeq_scored, "BAD")

NOTE: Added action set 'percentile'.
NOTE: Cloud Analytic Services made the uploaded file available as table SCOREDVALUES in caslib CASUSER(sorowl).
NOTE: The table SCOREDVALUES has been created in caslib CASUSER(sorowl) from binary data uploaded to Cloud Analytic Services.
NOTE: Added action set 'astore'.
NOTE: Cloud Analytic Services saved the file _26E50B4B870F46938BFDDD9A2.sashdat in caslib ModelStore.
NOTE: Added action set 'astore'.
NOTE: Cloud Analytic Services saved the file _55D32694629F4D339BD70180B.sashdat in caslib ModelStore.


### Python Model Management 
The function below will also generate the performance metadata and push the Python model into SAS Model Manager as well as generate score code for our Micro Analytic Service (MAS). (Note on the generated score code: it’s best to think of this as a starting point. Additional edits may be needed before publishing the Python model).

In [12]:
def mm_python(project_name, model_name, model, target, input_df, actual_df, desc, model_type):      
    ##################################################
    # CREATE OUTPUT INFORMATION  

    # Output folder name
    output_folder = 'Outputs'

    # Create Folder
    output_path = Path.cwd() / output_folder / model_name
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    ##################################################
    # CONNECT TO SAS VIYA
    host_session = 'https://' + hostname + '/'

    # Connect using a session 
    sess=Session(host_session, username, password, verify_ssl=False, protocol="http")

    ##################################################
    # COMPILE METADATA

    # Create JSON Files Object
    JSONFiles = pzmm.JSONFiles()

    # Write Input Variable JSON 
    JSONFiles.writeVarJSON(input_df, isInput=True, jPath=output_path)

    # Mock up Output Variables
    yCategory = actual_df.astype('str') # Ensuring dataframe length matches
    output_df = pd.DataFrame(columns=['EM_EVENTPROBABILITY', 'EM_CLASSIFICATION']) # Ensuring column names are what SAS expects
    output_df['EM_CLASSIFICATION'] = yCategory # Ensuring data type is nominal 
    output_df['EM_EVENTPROBABILITY'] = 0.5 # Ensuring data type is decimal  

    # Write Output Variable JSON
    JSONFiles.writeVarJSON(output_df, isInput=False, jPath=output_path)

    # Write File Metadata JSON
    JSONFiles.writeFileMetadataJSON(model_name, jPath=output_path)

    # Write Model Properties JSON
    JSONFiles.writeModelPropertiesJSON(modelName=model_name, 
        modelDesc=desc,
        targetVariable=target,
        modelType=model_type,
        modelPredictors=input_df.columns.array,
        targetEvent='1',
        numTargetCategories=2,
        eventProbVar='EM_EVENTPROBABILITY',
        jPath=output_path,
        modeler=username)

    # Get predictions
    trainProba = model.predict_proba(input_df)

    # Creating Assessment Data 
    trainData = pd.concat([pd.DataFrame(actual_df).reset_index(drop=True), pd.Series(data=trainProba[:,1])], axis=1)

    # Write Fit Statistics JSON
    JSONFiles.calculateFitStat(trainData=trainData, jPath=output_path)

    # Write ROC amd Lift Statistics JSON
    JSONFiles.generateROCLiftStat(target, 1, conn, trainData=trainData, jPath=output_path)

    ##################################################
    # PICKLE MODEL 
    pzmm.PickleModel.pickle_trained_model(_, model, model_name, output_path)

    ##################################################
    # WRITE SCORE CODE 

    # Generate Score Code Object
    ScoreCode = pzmm.ScoreCode()

    # Write Score Code
    ScoreCode.writeScoreCode(input_df, actual_df, model_name, '{}.predict_proba({})', model_name + '.pickle', pyPath=output_path)

    ##################################################
    # ZIP FOLDER
    zipIOFile = pzmm.ZipModel.zip_files(file_dir=output_path, model_prefix=model_name)

    ##################################################
    # PUSH ZIP FOLDER INTO MODEL MANAGER
    with sess: 
        modelRepo.import_model_from_zip(model_name, project_name, zipIOFile)

In [13]:
urllib3.disable_warnings()

# mm_python(project_name, model_name, model, scored_data, target, input_df, actual_df, desc, model_type)
mm_python('HMEQ Loan Default', 'HMEQ_XGBOOST', ml_pipe , 'BAD', hmeqdf, y, 'Generated in Jupyter Notebook', 'XGBOOST')

NOTE: Added action set 'percentile'.
NOTE: Cloud Analytic Services made the uploaded file available as table SCOREDVALUES in caslib CASUSER(sorowl).
NOTE: The table SCOREDVALUES has been created in caslib CASUSER(sorowl) from binary data uploaded to Cloud Analytic Services.


*** 
## Accessing Models via API
Now, we can head over to SAS Viya to manage our model using the visual interface. But, you don't need to use the visual interface in SAS Viya if you don't want to! You can actually access most of the model management capabilities through the SAS open source package SASCTL. For whatever isn't included in the open source package, you can do through API!

In this section, we will use the SAS Viya APIs to look at one of our deployed models. To use this notebook on your own, make sure you have deployed one of your models, either through the visual interface or programmatically. To get started, we need to generate an authorization token.

In [14]:
from requests import request
import urllib3
urllib3.disable_warnings()

# Get token
host = 'http://' + hostname 
url = host + '/SASLogon/oauth/token' 
r = request('POST', url,
            data='grant_type=password&username=%s&password=%s' %(username, password),
            headers={
                'Accept': 'application/json',
                'Content-Type': 'application/x-www-form-urlencoded'
            },
            auth=('sas.ec', ''),
            verify=False)
token = r.json()['access_token']

Let's pull the specific information for our deployed module.

In [15]:
deployed_name= input("Deployed Name:")

Deployed Name: hmeq_champ


In [16]:
headers = {'Authorization': 'Bearer ' + token}

url = host + '/microanalyticScore/modules/' + deployed_name + '/steps'

r = request('GET', url, params={}, headers = headers, verify=False)

r.json()

{'links': [{'method': 'GET',
   'rel': 'collection',
   'href': '/microanalyticScore/modules/hmeq_champ/steps',
   'uri': '/microanalyticScore/modules/hmeq_champ/steps',
   'type': 'application/vnd.sas.collection'},
  {'method': 'GET',
   'rel': 'self',
   'href': '/microanalyticScore/modules/hmeq_champ/steps?start=0&limit=20',
   'uri': '/microanalyticScore/modules/hmeq_champ/steps?start=0&limit=20',
   'type': 'application/vnd.sas.collection'},
  {'method': 'POST',
   'rel': '/microanalyticScore/modules/hmeq_champ/steps',
   'href': '/microanalyticScore/modules/hmeq_champ/steps',
   'uri': '/microanalyticScore/modules/hmeq_champ/steps',
   'type': 'application/vnd.sas.microanalytic.module.definition',
   'responseType': 'application/vnd.sas.microanalytic.module'}],
 'name': 'steps',
 'accept': 'application/vnd.sas.microanalytic.module.step',
 'count': 3,
 'items': [{'links': [{'method': 'GET',
     'rel': 'up',
     'href': '/microanalyticScore/modules/hmeq_champ/steps',
     'uri': 

To close out, let's use that information to score some data.

In [17]:
# Prepare payload
payload = '{"inputs":[{"name":"clage", "value": 12}, {"name":"clno", "value": 0}, {"name":"debtinc", "value": 25}, {"name":"delinq", "value": 1}, {"name":"derog", "value": 1},{"name":"job", "value": "Other"}, {"name":"loan", "value": 8000}, {"name":"mortdue", "value": 80000}, {"name":"ninq", "value": 0}, {"name":"reason", "value": "HomeImp"}, {"name":"value", "value": 100000}, {"name":"yoj", "value": 4}]}'
    
# Send request
headers = {'Content-Type': 'application/vnd.sas.microanalytic.module.step.input+json', 
               'Authorization': 'Bearer ' + token}
    
url = host + '/microanalyticScore/modules/' + deployed_name + '/steps/score'
        
r = request('POST', url, data=payload, headers = headers, verify=False)
r.json()

{'links': [],
 'version': 2,
 'moduleId': 'hmeq_champ',
 'stepId': 'score',
 'executionState': 'completed',
 'outputs': [{'name': 'EM_CLASSIFICATION', 'value': '           1'},
  {'name': 'EM_EVENTPROBABILITY', 'value': 0.8357853808167168},
  {'name': 'EM_PROBABILITY', 'value': 0.8357853808167168},
  {'name': 'I_BAD', 'value': '           1                    '},
  {'name': 'P_BAD0', 'value': 0.16421461918328317},
  {'name': 'P_BAD1', 'value': 0.8357853808167168},
  {'name': '_P_', 'value': None},
  {'name': '_WARN_', 'value': '    '}]}

In the JSON response above, we can pull out our model’s classification as well as probability of default. By utilizing APIs and JSON, we now can easily embed analytically models into other applications to aid in smart decision making. We were also able to get this response back in under a second. This rapid response allows us to quickly return analytical results to the systems that depend on them.  Analytics can be made more accessible one API at a time! 
***
## Conclusion
We have gone over a very simple modeling example using both Python and SAS analytics. We pushed both models into the SAS model repository and accessed a deployed model from API. 

In [18]:
conn.close()

***